In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import custom_projection_example
# from mpl_toolkits.basemap import Basemap


In [ ]:
eyedata = pd.read_csv('receptor_directions_buchner71.csv', header=None)
eyedata.rename(columns={'0':'x', '1':'y','2':'z'}, inplace=True)
eyedata.columns = ['x', 'y', 'z']
eyedata_buchner = eyedata.assign(left = np.concatenate((np.ones(699), np.zeros(699))))

In [ ]:
def wall_distance_stimulus(roof_height=2, stimulus_radius_outer=15, stimulus_radius_inner=0):
    stimulus_elevation = [(np.arctan2(stimulus_radius_inner, roof_height+1)*180/np.pi) % 360, 
                          (np.arctan2(stimulus_radius_outer, roof_height+1)*180/np.pi) % 360]
    
    return 1

In [ ]:
def convert_to_spherical(eyedata, twosided=False):
    elevation = np.array((np.arctan2(np.sqrt(eyedata['x']**2 + eyedata['y']**2), eyedata['z'])*180/np.pi) % 360)
    if twosided:
        azimuth = np.array((np.arctan2(eyedata['y'], eyedata['x'])*180/np.pi))
    else:
        azimuth = np.array((np.arctan2(eyedata['y'], eyedata['x'])*180/np.pi) % 360)
    return elevation, azimuth

def ommatidia_range(eyedata, roof_height=2, stimulus_radius_outer=15, stimulus_radius_inner=0, frontal_gap=0, only_frontal_extension=None, elevation_range=None):
    eyedata_left = eyedata.loc[eyedata['left'] == 1]
    eyedata_right = eyedata.loc[eyedata['left'] != 1]
    elevation_left, azimuth_left = convert_to_spherical(eyedata_left)
    elevation_right, azimuth_right = convert_to_spherical(eyedata_right)
    if not elevation_range:
        stimulus_elevation = [(np.arctan2(stimulus_radius_inner, roof_height+1)*180/np.pi) % 360, (np.arctan2(stimulus_radius_outer, roof_height+1)*180/np.pi) % 360]
    else:
        stimulus_elevation = elevation_range
    if not only_frontal_extension:
        stimulus_azimuth = [0+frontal_gap, 360-frontal_gap]
        stimulus_ommatidia_left = np.intersect1d(np.argwhere((stimulus_elevation[0] < elevation_left) & (elevation_left < stimulus_elevation[1])).flatten(), 
                                             np.argwhere((stimulus_azimuth[0] < azimuth_left) & (azimuth_left < stimulus_azimuth[1])).flatten())
        stimulus_ommatidia_right = np.intersect1d(np.argwhere((stimulus_elevation[0] < elevation_right) & (elevation_right < stimulus_elevation[1])).flatten(), 
                                             np.argwhere((stimulus_azimuth[0] < azimuth_right) & (azimuth_right < stimulus_azimuth[1])).flatten())
    else:
        stimulus_azimuth = [0+only_frontal_extension, 360-only_frontal_extension]
        stimulus_ommatidia_left = np.intersect1d(np.argwhere((stimulus_elevation[0] < elevation_left) & (elevation_left < stimulus_elevation[1])).flatten(), 
                                             np.argwhere((stimulus_azimuth[0] > azimuth_left) | (azimuth_left > stimulus_azimuth[1])).flatten())
        stimulus_ommatidia_right = np.intersect1d(np.argwhere((stimulus_elevation[0] < elevation_right) & (elevation_right < stimulus_elevation[1])).flatten(), 
                                             np.argwhere((stimulus_azimuth[0] > azimuth_right) | (azimuth_right > stimulus_azimuth[1])).flatten())
    


    n_ommatidia_left = stimulus_ommatidia_left.shape[0]/699
    n_ommatidia_right = stimulus_ommatidia_right.shape[0]/699
    return n_ommatidia_left, n_ommatidia_right, stimulus_ommatidia_left, stimulus_ommatidia_right


In [ ]:
# get RF data
Dir = r'path\to\receptive_field_data'
Dir_output = r'path\to\destination_folder'
cellname = 'HSE'
strain = 'FlpND'
data = mat73.loadmat(os.path.join(Dir, 'scanning_rect_grp_' + cellname + '_' + strain + '.mat'))
RF = data['grp_data']['meanRF']
grid = data['grp_data']['deformed_grid']
RF_amp = data['grp_data']['RF_ampmean']
expansion_factor=5
RF_amp_expanded = zoom(RF_amp, [expansion_factor, expansion_factor])
grid_new = expand_grid(13*expansion_factor, 9*expansion_factor)
x_new = grid_new[:, :, 0]
y_new = grid_new[:, :, 1]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))
eyedata_list = [eyedata_buchner]
eyedata_title = ['Buchner et al. 1971']
for i in range(len(eyedata_list)):
    eyedata = eyedata_list[i]
    eyedata_left = eyedata.loc[eyedata['left'] == 1]
    eyedata_right = eyedata.loc[eyedata['left'] != 1]
    elevation_left, azimuth_left = convert_to_spherical(eyedata_left, twosided=True)
    elevation_right, azimuth_right = convert_to_spherical(eyedata_right, twosided=True)

    # ax.scatter(azimuth_left, elevation_left, alpha=0.6, s=20, c='g', linewidths=0)
    # ax.scatter(azimuth_right, elevation_right, alpha=0.6, s=20, c='r', linewidths=0)

    n_ommatidia_left, n_ommatidia_right, stimulus_ommatidia_left, stimulus_ommatidia_right = \
    ommatidia_range(eyedata, roof_height=2, stimulus_radius_outer=30, stimulus_radius_inner=0, frontal_gap=0, only_frontal_extension=70, elevation_range=[90-65, 180-65])

    stimulus_ommatidia_left = np.in1d(np.arange(699), stimulus_ommatidia_left)
    stimulus_ommatidia_right = np.in1d(np.arange(699), stimulus_ommatidia_right)

    ax.scatter(azimuth_left[~stimulus_ommatidia_left], elevation_left[~stimulus_ommatidia_left], alpha=0.5, s=20, facecolors='none', edgecolors='g', linewidths=1)
    ax.scatter(azimuth_right[~stimulus_ommatidia_right], elevation_right[~stimulus_ommatidia_right], alpha=0.5, s=20, facecolors='none', edgecolors='r', linewidths=1)

    ax.scatter(azimuth_left[stimulus_ommatidia_left], elevation_left[stimulus_ommatidia_left], alpha=0.5, c='g', label='stimulus', s=18, linewidths=0)
    ax.scatter(azimuth_right[stimulus_ommatidia_right], elevation_right[stimulus_ommatidia_right], alpha=0.5, c='r', s=18, linewidths=0)

    # ##ephys
    # ax.fill_betweenx(y=[90-65, 180-65], x1=-70, x2=70, color='coral', alpha=0.4, label='Electrophysiology', lw=0)
    # ##behaviour
    # ax.fill_betweenx(y=[0, 90-10], x1=-180, x2=180, color='olive', alpha=0.4, label='Behaviour', lw=0)

    ## RF contour
    # convert to degrees
    x_new = x_new*180/np.pi * -1
    y_new = y_new*180/np.pi * -1
    y_new = y_new + 30
    ax.contourf(x_new, y_new, RF_amp_expanded/np.amax(RF_amp_expanded), levels=[0.3, 0.6, 1], vmin=0, vmax=1, algorithm='serial',
                cmap=ListedColormap(create_color_gradient(color_dict['FlpND'][0], [1, 1, 1], 128)), alpha=0.2, antialiased=False)

    ax.set_ylim(185, -5)
    ax.set_xlim(-181, 181)
    ax.set_yticks(np.arange(0, 181, 30))
    ax.set_xticks(np.arange(-180, 181, 45))
    ax.set_yticklabels([str(x) for x in np.arange(-90, 91, 30)])
    ax.set_title(eyedata_title[i])
    ax.set_aspect(1.5)
    ax.legend()
# fig.savefig('ommatidia_range3.pdf', dpi=600, transparent=True, bbox_inches='tight')


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(30, 10))
eyedata_list = [eyedata_buchner, eyedata_2019, eyedata_2023]
eyedata_title = ['Buchner et al. 1971', 'Reiser 2019', 'Reiser 2023']
map = Basemap(projection='moll',lat_0=0,lon_0=0, rsphere=1)
for i in range(len(eyedata_list)):
    eyedata = eyedata_list[i]
    eyedata_left = eyedata.loc[eyedata['left'] == 1]
    eyedata_right = eyedata.loc[eyedata['left'] != 1]
    elevation_left, azimuth_left = convert_to_spherical(eyedata_left, twosided=True)
    elevation_left = elevation_left - 90
    elevation_right, azimuth_right = convert_to_spherical(eyedata_right, twosided=True)
    elevation_right = elevation_right - 90
    x_left, y_left = map(azimuth_left, elevation_left)
    x_right, y_right = map(azimuth_right, elevation_right)
    ax[i].scatter(x_left, y_left, alpha=0.5, c='g', s=20)
    ax[i].scatter(x_right, y_right, alpha=0.5, c='r', s=20)

    n_ommatidia_left, n_ommatidia_right, stimulus_ommatidia_left, stimulus_ommatidia_right = \
    ommatidia_range(eyedata, roof_height=2, stimulus_radius_outer=30, stimulus_radius_inner=0, frontal_gap=0)

    ax[i].scatter(x_left[stimulus_ommatidia_left], y_left[stimulus_ommatidia_left], alpha=0.5, c='k', label='stimulus', s=15)
    ax[i].scatter(x_right[stimulus_ommatidia_right], y_right[stimulus_ommatidia_right], alpha=0.5, c='k', s=15)
    # ax[i].set_xlim(-181, 181)
    ax[i].set_title(eyedata_title[i])
    ax[i].set_aspect(1)
    
    map.drawmapboundary(fill_color='w', ax=ax[i])
    map.drawparallels(np.arange(-90, 90, 30), labels=[1,0,0,0], ax=ax[i], linewidth=2)
    map.drawmeridians([-90, -45, 45, 90], ax=ax[i], linewidth=2)
    ax[i].set_ylim(np.pi, 0)
    

In [ ]:
# projection = "sinusoidal"
# fig, ax = plt.subplots(1, 1, figsize=(30, 10), subplot_kw={'projection': projection})
# eyedata_list = [eyedata_buchner, eyedata_2019, eyedata_2023]
# eyedata_title = ['Buchner et al. 1971', 'Reiser 2019', 'Reiser 2023']

projection = "mollweide"
fig, ax = plt.subplots(1, 1, figsize=(6,6), subplot_kw={'projection': projection})
eyedata_list = [eyedata_buchner]
eyedata_title = ['Buchner et al. 1971']
# map = Basemap(projection='moll',lat_0=0,lon_0=0, rsphere=1)
for i in range(len(eyedata_list)):
    eyedata = eyedata_list[i]
    eyedata_left = eyedata.loc[eyedata['left'] == 1]
    eyedata_right = eyedata.loc[eyedata['left'] != 1]
    elevation_left, azimuth_left = convert_to_spherical(eyedata_left, twosided=True)
    elevation_left = (elevation_left - 90)*(np.pi/180)*-1
    azimuth_left = (azimuth_left)*(np.pi/180)
    elevation_right, azimuth_right = convert_to_spherical(eyedata_right, twosided=True)
    elevation_right = (elevation_right - 90)*(np.pi/180)*-1
    azimuth_right = (azimuth_right)*(np.pi/180)

    ax.scatter(azimuth_left, elevation_left, alpha=0.5, c='g', s=10, linewidths=0)
    ax.scatter(azimuth_right, elevation_right, alpha=0.5, c='r', s=10, linewidths=0)

    # n_ommatidia_left, n_ommatidia_right, stimulus_ommatidia_left, stimulus_ommatidia_right = \
    # ommatidia_range(eyedata, roof_height=2, stimulus_radius_outer=30, stimulus_radius_inner=0, frontal_gap=0)
    ##ephys
    # n_ommatidia_left, n_ommatidia_right, stimulus_ommatidia_left, stimulus_ommatidia_right = \
    # ommatidia_range(eyedata, roof_height=2, stimulus_radius_outer=30, stimulus_radius_inner=0, frontal_gap=0, only_frontal_extension=70, elevation_range=[90-65, 180-65])

    # stimulus_ommatidia_left = np.in1d(np.arange(699), stimulus_ommatidia_left)
    # stimulus_ommatidia_right = np.in1d(np.arange(699), stimulus_ommatidia_right)

    # ax.scatter(azimuth_left[~stimulus_ommatidia_left], elevation_left[~stimulus_ommatidia_left], alpha=0.5, s=10, facecolors='none', edgecolors='g', linewidths=0.5)
    # ax.scatter(azimuth_right[~stimulus_ommatidia_right], elevation_right[~stimulus_ommatidia_right], alpha=0.5, s=10, facecolors='none', edgecolors='r', linewidths=0.5)

    # ax.scatter(azimuth_left[stimulus_ommatidia_left], elevation_left[stimulus_ommatidia_left], alpha=0.5, c='g', label='stimulus', s=9, linewidths=0)
    # ax.scatter(azimuth_right[stimulus_ommatidia_right], elevation_right[stimulus_ommatidia_right], alpha=0.5, c='r', s=9, linewidths=0)

    ax.set_title(eyedata_title[i])

    ## visual field
    # ax.fill_betweenx(y=[-np.pi/2, np.pi/2], x1=(160/180)*-np.pi, x2=(160/180)*np.pi, color=(219/255, 228/255, 231/255), alpha=0.5, label='Behaviour', lw=0)
    ##ephys
    # ax.fill_betweenx(y=[(25/90) * -np.pi/2, (65/90) * np.pi/2], x1=(70/90)*-np.pi/2, x2=(70/90)*np.pi/2, color='g', alpha=0.4, label='Electrophysiology', lw=0)
    ##behaviour
    ax.fill_betweenx(y=[(10/90) * np.pi/2, np.pi/2], x1=-np.pi, x2=np.pi, color='goldenrod', alpha=0.2, label='Behaviour', lw=0)

    ax.set_xticks(np.arange(-np.pi, np.pi+0.1, np.pi/4))
    ax.set_yticks(np.arange(-np.pi/2, np.pi/2+0.1, np.pi/6))
    ax.grid(visible=True, linewidth=0.5, color='grey', alpha=0.5)
    # plt.box(False)

    # ## RF contour
    grid_new = expand_grid_spherical(13*expansion_factor, 9*expansion_factor)
    x_new = grid_new[:, :, 0]
    y_new = grid_new[:, :, 1]
    x_new = x_new * -1
    y_new = y_new + ((20/180)*np.pi)
    ax.contourf(x_new, y_new, RF_amp_expanded/np.amax(RF_amp_expanded), levels=[0.3, 1], vmin=0, vmax=1, algorithm='serial',
            cmap=ListedColormap(create_color_gradient([0, 1, 0], [1, 1, 1], 128)), alpha=0.4, antialiased=False)
    ax.contourf(x_new * -1, y_new, RF_amp_expanded/np.amax(RF_amp_expanded), levels=[0.3, 1], vmin=0, vmax=1, algorithm='serial',
            cmap=ListedColormap(create_color_gradient([1, 0, 0], [1, 1, 1], 128)), alpha=0.4, antialiased=False)

    fig.savefig('behaviour_stimulus_mollweiede_projection'+cellname+strain+'.pdf', dpi=600, transparent=True, bbox_inches='tight')
